In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd "drive/MyDrive/Segmentation"

In [ ]:
images_per_gui = 5000
out_dir = "dataset"
out_dir_img = out_dir+"/images"
out_dir_mask = out_dir+"/masks"
gui_path = "dataset_generator/empty_guis"
textures_path = "dataset_generator/textures"
backgrounds_path = "dataset_generator/backgrounds"
texture_width = 64
texture_height = 64
inv_x = 928
inv_y = 376
output_width = 640
output_height = 360

In [ ]:
from PIL import Image
import os
import random
import numpy as np

In [ ]:
for dir in [out_dir, out_dir_img, out_dir_mask]:
    os.makedirs(dir, exist_ok=True)

textures = []
for texture in os.listdir(textures_path):
    if not os.path.isfile(f"{textures_path}/{texture}") or not texture.endswith(".png"):
        continue
    textures.append(Image.open(f"{textures_path}/{texture}").convert("RGBA").resize((texture_width, texture_height), Image.NEAREST))

backgrounds = []
for background_file in os.listdir(backgrounds_path):
    backgrounds.append(Image.open(f"{backgrounds_path}/{background_file}").convert("RGB"))


count = 0
for gui_file in os.listdir(gui_path):
    if not os.path.isfile(f"{gui_path}/{gui_file}") or not gui_file.endswith(".png"):
        continue
    gui_coords = gui_file.replace(".png", ".txt")
    if not os.path.isfile(f"{gui_path}/{gui_coords}"):
        continue
    with open(f"{gui_path}/{gui_coords}") as f:
        coords = f.readlines()
    gui_img = Image.open(f"{gui_path}/{gui_file}").convert("RGBA")
    gui_file = gui_file.replace(".png", "")
    for _ in range(images_per_gui):
        count += 1
        count_str = str(count).zfill(12)
        background = random.sample(backgrounds, 1)[0].copy()
        new_img = gui_img.copy()
        mask = np.zeros((background.size[1], background.size[0]), dtype=np.uint8)
        for coord in coords:
            if random.randrange(2) == 0:
                continue
            x, y = list(map(lambda x: int(x), coord.replace("\n", "").split()))
            texture = random.sample(textures, 1)[0]
            new_img.paste(texture, (x, y), mask=texture)
            for mask_y in range(texture.height):
                for mask_x in range(texture.width):
                    if texture.getpixel((mask_x, mask_y))[3] == 0:
                        continue
                    mask[y+mask_y+inv_y][x+mask_x+inv_x] = 255
        Image.fromarray(mask, "L").save(f"{out_dir_mask}/{count_str}.png", "PNG")
        background.paste(new_img, (inv_x, inv_y), mask=new_img)
        background.save(f"{out_dir_img}/{count_str}.png", "PNG")
        if count % 100 == 0:
            print(f"saved files {count-100} to {count}")
print("FINISHED")